# 금융정보
> * 코스피 지수
> * 코스닥 지수
> * 달러/원화 환율  

기간 : 2017년 12월 01일 ~

In [1]:
def Get_Finance_Info(target, symbol, date, nowdate):
    from pandas_datareader import DataReader
    print (target, symbol, date, nowdate)
    value = True
    while value:
        try:
            df = DataReader(symbol, "yahoo", date, nowdate)
        except:
            value = True
        else:
            value = False
    return df.add_suffix('_'+target)

def Get_Finance(date): 
    import pandas as pd
    import chat_bot as cb
    from datetime import datetime
    nowdate = datetime.now().date()
    date = datetime.strptime(str(date),'%Y%m%d').date()
    target = {'KOSPI' : '^KS11',
             'KOSDAQ' : '^KQ11',
             'CURRENCY' : 'KRW=X'}
    df = list(map(lambda x: Get_Finance_Info(x, target[x], date, nowdate), target))
    df = pd.concat(df, axis = 1)
    df = df.dropna()
    df.reset_index(inplace = True)
    df['Date'] = df.Date.dt.strftime('%Y.%m.%d')
    slack = cb.Slacker(cb.slacktoken())
    strNowDate = nowdate.strftime('%Y-%m-%d')
    slack.chat.post_message('# notification',
                            '''targetData : {}, {}, {}
                            from : {}
                            to : {} '''.format(*target, date, strNowDate))
    return df

from datetime import datetime
import Database_Handler as dh
startTime = datetime.now()
dateIs = startTime.date().strftime('%Y%m%d')
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
df_finance = Get_Finance(dateIs)
df_finance.drop_duplicates(subset='Date', inplace = True)
financeCollectionName = 'Finance2'
useCollection_Finance = dh.Use_Collection(useDb, financeCollectionName)
useCollection_Finance.insert_many(df_finance.to_dict('records'))
endTime = datetime.now()
runningTime= endTime - startTime
print (runningTime)

KOSPI ^KS11 2018-02-03 2018-02-03
KOSDAQ ^KQ11 2018-02-03 2018-02-03
CURRENCY KRW=X 2018-02-03 2018-02-03
0:00:04.348348
